# ML Foreclosure Binary Classification - EPOCH1 5% sample dataset
Only "resolved" outcomes - Either paid off or default 

    target = zeroBalCode (set to object dtype) 
    Binary classification sets zeroBalCode to either "0" or "1" for the 7 possible outcomes
    
    "0" = "Loan Success"
        * 01 = Prepaid or Matured
        * 06 = Repurchased
        * 16 = Reperforming Loan Sale

    "1" =  "Loan Failure" - Most likely loss of money (but not necessarily) one of below conditions:
        * 02 = Third Party Sale
        * 03 = Short Sale
        * 09 = Deed-in-Lieu,REO
        * 15 = Note Sale


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#!pip install pycaret
from imblearn.over_sampling import RandomOverSampler
from pycaret.classification import *
#from pycaret.regression import *
pd.set_option('display.max_columns', None)


# Importing the data - don't use *.ML.csv files!!!
If you have *.MLReady.csv then [click here](#MLReady)

In [2]:
df=pd.read_csv("data/FM_Acq2_Perf_FMAC_EPOCH1_loanAge_5pct.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,id,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origDate,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore,loanAge,monMatur,mSA,zeroBalCode,deliqGood,deliqBad,deliqMax,fmacRateMax,fmacRateMin,fmacRateAvg,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct
0,349870,350665,381717688575,R,OTHER,6.875,94000,180,2001-08-01,69.0,69.0,1.0,62.0,C,NH,31,0.0,0.0,691.0,691.0,691.0,33,147.0,31700,1.0,24,7,1.0,7.00,6.91,6.9500,0.013025,5.11,-0.125,0.035,-0.0750,-0.017857,0.005065,-0.010791
1,1426378,1427173,337308630846,C,"CITIMORTGAGE, INC.",5.750,235000,360,2004-01-01,80.0,80.0,1.0,20.0,P,CA,945,0.0,0.0,785.0,785.0,785.0,13,347.0,41860,1.0,11,1,1.0,5.87,5.64,5.7125,0.040780,4.27,-0.120,-0.110,0.0375,-0.020443,-0.019504,0.006565
2,1920339,1921134,930608058483,B,SUNTRUST MORTGAGE INC.,6.875,168000,360,2006-07-01,72.0,90.0,1.0,39.0,R,OR,975,0.0,0.0,661.0,661.0,661.0,35,325.0,32780,9.0,21,13,11.0,6.80,6.72,6.7625,0.011905,5.15,0.075,-0.155,0.1125,0.011029,-0.023065,0.016636
3,365256,366051,434312797171,R,"CAPITAL ONE, NATIONAL ASSOCIATION",5.750,210000,180,2001-11-01,33.0,33.0,2.0,41.0,C,LA,704,0.0,0.0,742.0,735.0,738.5,70,110.0,35380,1.0,68,2,1.0,7.02,6.45,6.6580,0.088372,4.24,-1.270,0.700,-0.9080,-0.180912,0.108527,-0.136377
4,1152850,1153645,546668906438,C,"CITIMORTGAGE, INC.",4.375,195000,180,2003-06-01,53.0,53.0,1.0,25.0,R,WA,980,0.0,0.0,768.0,768.0,768.0,133,47.0,42660,1.0,131,1,1.0,5.26,5.21,5.2300,0.009597,3.37,-0.885,0.835,-0.8550,-0.168251,0.160269,-0.163480


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113918 entries, 0 to 113917
Data columns (total 39 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          113918 non-null  int64  
 1   Unnamed: 0.1        113918 non-null  int64  
 2   id                  113918 non-null  int64  
 3   origChannel         113918 non-null  object 
 4   sellerName          113918 non-null  object 
 5   origIntRate         113918 non-null  float64
 6   origUPB             113918 non-null  int64  
 7   origLoanTerm        113918 non-null  int64  
 8   origDate            113918 non-null  object 
 9   origLTV             113918 non-null  float64
 10  origCLTV            113918 non-null  float64
 11  numBorrowers        113918 non-null  float64
 12  origDebtIncRatio    113918 non-null  float64
 13  loanPurp            113918 non-null  object 
 14  propState           113918 non-null  object 
 15  zipCode             113918 non-nul

## Remove columns not involved in 'Foreclosure or Not' query

In [4]:
df.drop(['Unnamed: 0','Unnamed: 0.1','id','fredRate',\
         'bestCreditScore','worstCreditScore','monMatur',\
         'deliqGood','deliqBad','deliqMax',\
        'fmacRateMax','fmacRateMin','fmacRateVolatility',\
        'rateDiffAbove','rateDiffBelow','rateDiffAbovePct','rateDiffBelowPct',\
        ],1,inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113918 entries, 0 to 113917
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   origChannel       113918 non-null  object 
 1   sellerName        113918 non-null  object 
 2   origIntRate       113918 non-null  float64
 3   origUPB           113918 non-null  int64  
 4   origLoanTerm      113918 non-null  int64  
 5   origDate          113918 non-null  object 
 6   origLTV           113918 non-null  float64
 7   origCLTV          113918 non-null  float64
 8   numBorrowers      113918 non-null  float64
 9   origDebtIncRatio  113918 non-null  float64
 10  loanPurp          113918 non-null  object 
 11  propState         113918 non-null  object 
 12  zipCode           113918 non-null  int64  
 13  pMIperct          113918 non-null  float64
 14  mortInsType       113918 non-null  float64
 15  avgCreditScore    113918 non-null  float64
 16  loanAge           11

### Check if any columns have a null value

In [6]:
df.apply(lambda x: x.isnull().sum(), axis=0)

origChannel         0
sellerName          0
origIntRate         0
origUPB             0
origLoanTerm        0
origDate            0
origLTV             0
origCLTV            0
numBorrowers        0
origDebtIncRatio    0
loanPurp            0
propState           0
zipCode             0
pMIperct            0
mortInsType         0
avgCreditScore      0
loanAge             0
mSA                 0
zeroBalCode         0
fmacRateAvg         0
rateDiffAvg         0
rateDiffAvgPct      0
dtype: int64

## Engineer originDate into Date features

In [7]:
df['origDate'] = pd.to_datetime(df['origDate'])

In [8]:
df['origYear'] = df['origDate'].dt.year
df['origMonth'] = df['origDate'].dt.month

In [9]:
df.drop(['origDate'],1,inplace=True)

In [10]:
df[['origYear','origMonth']].head()

,origYear,origMonth
0,2001,8
1,2004,1
2,2006,7
3,2001,11
4,2003,6


# Turn zero Balance Code Into binary "0" GOOD and "1" BAD

In [11]:
df.zeroBalCode.unique()

array([ 1.,  9.,  3.,  6.,  2., 16., 15.])

In [12]:
df['zeroBalCode'] = df.zeroBalCode.replace([1,6,16],0)
df['zeroBalCode'] = df.zeroBalCode.replace([9,3,2,15],1)

In [13]:
df.zeroBalCode.unique()

array([0., 1.])

In [14]:
#Checking datatypes of individual feature
df.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,avgCreditScore,loanAge,mSA,zeroBalCode,fmacRateAvg,rateDiffAvg,rateDiffAvgPct,origYear,origMonth
0,R,OTHER,6.875,94000,180,69.0,69.0,1.0,62.0,C,NH,31,0.0,0.0,691.0,33,31700,0.0,6.9500,-0.0750,-0.010791,2001,8
1,C,"CITIMORTGAGE, INC.",5.750,235000,360,80.0,80.0,1.0,20.0,P,CA,945,0.0,0.0,785.0,13,41860,0.0,5.7125,0.0375,0.006565,2004,1
2,B,SUNTRUST MORTGAGE INC.,6.875,168000,360,72.0,90.0,1.0,39.0,R,OR,975,0.0,0.0,661.0,35,32780,1.0,6.7625,0.1125,0.016636,2006,7
3,R,"CAPITAL ONE, NATIONAL ASSOCIATION",5.750,210000,180,33.0,33.0,2.0,41.0,C,LA,704,0.0,0.0,738.5,70,35380,0.0,6.6580,-0.9080,-0.136377,2001,11
4,C,"CITIMORTGAGE, INC.",4.375,195000,180,53.0,53.0,1.0,25.0,R,WA,980,0.0,0.0,768.0,133,42660,0.0,5.2300,-0.8550,-0.163480,2003,6


# Review values for dates and slice (if needed) the timeframe you want

In [15]:
df.origYear.unique()

array([2001, 2004, 2006, 2003, 2007, 2002, 2005, 2000, 2008, 1999],
      dtype=int64)

In [16]:
df['origMonth'].unique()

array([ 8,  1,  7, 11,  6,  5,  2,  4,  3, 10, 12,  9], dtype=int64)

In [17]:
df.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,avgCreditScore,loanAge,mSA,zeroBalCode,fmacRateAvg,rateDiffAvg,rateDiffAvgPct,origYear,origMonth
0,R,OTHER,6.875,94000,180,69.0,69.0,1.0,62.0,C,NH,31,0.0,0.0,691.0,33,31700,0.0,6.9500,-0.0750,-0.010791,2001,8
1,C,"CITIMORTGAGE, INC.",5.750,235000,360,80.0,80.0,1.0,20.0,P,CA,945,0.0,0.0,785.0,13,41860,0.0,5.7125,0.0375,0.006565,2004,1
2,B,SUNTRUST MORTGAGE INC.,6.875,168000,360,72.0,90.0,1.0,39.0,R,OR,975,0.0,0.0,661.0,35,32780,1.0,6.7625,0.1125,0.016636,2006,7
3,R,"CAPITAL ONE, NATIONAL ASSOCIATION",5.750,210000,180,33.0,33.0,2.0,41.0,C,LA,704,0.0,0.0,738.5,70,35380,0.0,6.6580,-0.9080,-0.136377,2001,11
4,C,"CITIMORTGAGE, INC.",4.375,195000,180,53.0,53.0,1.0,25.0,R,WA,980,0.0,0.0,768.0,133,42660,0.0,5.2300,-0.8550,-0.163480,2003,6


# Coerce the column dtype (int in this case)

In [18]:
df = df.astype({'origLTV':'int','origCLTV':'int','numBorrowers':'int','origDebtIncRatio':'int',\
               'mortInsType':'int','loanAge':'int',\
               'avgCreditScore':'int','zeroBalCode':'int'})

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113918 entries, 0 to 113917
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   origChannel       113918 non-null  object 
 1   sellerName        113918 non-null  object 
 2   origIntRate       113918 non-null  float64
 3   origUPB           113918 non-null  int64  
 4   origLoanTerm      113918 non-null  int64  
 5   origLTV           113918 non-null  int32  
 6   origCLTV          113918 non-null  int32  
 7   numBorrowers      113918 non-null  int32  
 8   origDebtIncRatio  113918 non-null  int32  
 9   loanPurp          113918 non-null  object 
 10  propState         113918 non-null  object 
 11  zipCode           113918 non-null  int64  
 12  pMIperct          113918 non-null  float64
 13  mortInsType       113918 non-null  int32  
 14  avgCreditScore    113918 non-null  int32  
 15  loanAge           113918 non-null  int32  
 16  mSA               11

<a id='MLReady'></a>

# Train Test Split to create holdout dataset outside of PyCaret

In [20]:
good = df.zeroBalCode.value_counts()[0]
bad = df.zeroBalCode.value_counts()[1]
perct_bad = round(bad/(good + bad)*100,2)
print(f'We have {perct_bad}% Foreclosures in our dataset')

We have 15.32% Foreclosures in our dataset


In [21]:
from sklearn.model_selection import train_test_split

training_features, test_features, \
training_target, test_target, = train_test_split(
    df.drop(['zeroBalCode'], axis=1)
    , df['zeroBalCode']
    , test_size = .1
    , random_state=12
)

In [22]:
# Further split the training data into training/test
x_train, x_val, y_train, y_val = train_test_split(
    training_features
    , training_target
    , test_size = .1
    ,random_state=12
)

In [23]:
# For the training data, randomly sample 
# ros = RandomOverSampler(sampling_strategy='minority')
# x_train_res, y_train_res = ros.fit_sample(x_train, y_train)

In [24]:
print('#############################################')
print('Before oversampling: "Closed" crushes "Default" and causes issues:')
print(training_target.value_counts())
# print('')
# print('Before oversampling: "Closed" and "Default" are equal')
# print(y_train_res.value_counts())

#############################################
Before oversampling: "Closed" crushes "Default" and causes issues:
0    86858
1    15668
Name: zeroBalCode, dtype: int64


# Recombine X_train_res, y_train_res so PyCaret can deal with one df

In [25]:
y_train.shape

(92273,)

In [26]:
df = x_train.copy()

In [27]:
df['zeroBalCode'] = y_train

In [28]:
df.shape

(92273, 23)

# Review metrics last time before run (to have record of starting values)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92273 entries, 98148 to 64256
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   origChannel       92273 non-null  object 
 1   sellerName        92273 non-null  object 
 2   origIntRate       92273 non-null  float64
 3   origUPB           92273 non-null  int64  
 4   origLoanTerm      92273 non-null  int64  
 5   origLTV           92273 non-null  int32  
 6   origCLTV          92273 non-null  int32  
 7   numBorrowers      92273 non-null  int32  
 8   origDebtIncRatio  92273 non-null  int32  
 9   loanPurp          92273 non-null  object 
 10  propState         92273 non-null  object 
 11  zipCode           92273 non-null  int64  
 12  pMIperct          92273 non-null  float64
 13  mortInsType       92273 non-null  int32  
 14  avgCreditScore    92273 non-null  int32  
 15  loanAge           92273 non-null  int32  
 16  mSA               92273 non-null  in

In [30]:
df.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,avgCreditScore,loanAge,mSA,fmacRateAvg,rateDiffAvg,rateDiffAvgPct,origYear,origMonth,zeroBalCode
98148,R,FLEET NATIONAL BANK,7.250,80000,360,40,40,1,49,P,NJ,73,0.0,0,741,122,35620,7.0325,0.2175,0.030928,2001,1,0
72477,R,"JPMORGAN CHASE BANK, NA",5.750,275000,360,76,76,2,41,C,CA,917,0.0,0,677,108,40140,5.6350,0.1150,0.020408,2004,2,0
39555,C,OTHER,6.250,125000,360,71,71,2,25,P,NJ,80,0.0,0,778,85,35620,5.8760,0.3740,0.063649,2003,12,0
93412,R,OTHER,6.000,143000,360,84,84,2,33,R,PA,170,12.0,1,722,108,25420,5.7450,0.2550,0.044386,2003,3,0
37859,B,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",5.625,82000,180,63,63,1,20,C,SC,293,0.0,0,646,161,43900,6.0680,-0.4430,-0.073006,2002,11,0


# Pycaret - Setup with categorical definition

### Use complex setup 

In [31]:
model_setup = setup(
    df
    , target = 'zeroBalCode' # PyCaret will list this as "Label"
    , normalize = True
    , transformation = False
    , pca = False 
    , ignore_low_variance = False # Variance is calculated using the ratio of unique values to the number of samples, and the ratio of the most common value to the frequency of the second most common value. 
    , ignore_features = None
    , handle_unknown_categorical = True
    , remove_outliers = True
    , remove_multicollinearity = False
    , polynomial_features = True
    , trigonometry_features = False
    , feature_selection = True
    , feature_interaction = False
    , fix_imbalance = True
    , silent = True
    , profile = False
    # outliers from the training data are removed using PCA linear dimensionality reduction using the Singular Value Decomposition technique.
    , bin_numeric_features = [
        'avgCreditScore'
        ] # Set to True to bin numerics using K Means
    , categorical_features = [
         'origChannel'
        , 'loanPurp'
        , 'origYear'
        , 'origMonth'
        , 'mortInsType'
        , 'origLoanTerm'
        ]
    , high_cardinality_features = [
         'propState'
        , 'sellerName'
        , 'mSA'
        , 'zipCode'
    ]
    , high_cardinality_method = 'clustering'
    , numeric_features = [
        'origIntRate'
        , 'numBorrowers'
        , 'origUPB'
        , 'origLTV'
        , 'origCLTV'
        , 'pMIperct'
        , 'loanAge'
        , 'origDebtIncRatio'
        , 'avgCreditScore'
        , 'rateDiffAvg'
        , 'rateDiffAvgPct'
        , 'fmacRateAvg'
    ]
)

Setup Succesfully Completed!


,Description,Value
0,session_id,8498
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(92273, 23)"
4,Missing Values,False
5,Numeric Features,14
6,Categorical Features,8
7,Ordinal Features,False
8,High Cardinality Features,True
9,High Cardinality Method,clustering


## Compare Models with no blacklist exclusions

In [32]:
model_results=compare_models(sort='Recall') #blacklist = ['tr','ransac'],'lar','par','huber','llar','lasso','en','ridge','omp','br','svm'])
model_results

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,SVM - Linear Kernel,0.7084,0.0000,0.7553,0.3109,0.4397,0.2870,0.3391,0.6020
1,Ridge Classifier,0.7168,0.0000,0.7540,0.3166,0.4460,0.2961,0.3464,0.1058
2,Linear Discriminant Analysis,0.7168,0.7958,0.7540,0.3166,0.4459,0.2961,0.3464,0.7604
3,Naive Bayes,0.5945,0.7324,0.7524,0.2361,0.3594,0.1679,0.2285,0.0851
4,Logistic Regression,0.7221,0.7960,0.7448,0.3199,0.4475,0.2994,0.3471,1.0100
5,K Neighbors Classifier,0.6543,0.7182,0.6949,0.2596,0.3780,0.2025,0.2496,2.4471
6,Quadratic Discriminant Analysis,0.6191,0.6930,0.6878,0.2398,0.3545,0.1680,0.2145,0.3144
7,Ada Boost Classifier,0.7791,0.7763,0.5333,0.3490,0.4219,0.2926,0.3027,7.3288
8,Gradient Boosting Classifier,0.8186,0.7965,0.4464,0.4083,0.4264,0.3189,0.3194,32.0201
9,Decision Tree Classifier,0.7748,0.6086,0.3704,0.3009,0.3320,0.1984,0.1999,2.0117


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=8498, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

## Create models 

### LDA Classifier

In [36]:
lda = create_model('lda', fold=10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7124,0.7933,0.7565,0.3133,0.4430,0.2915,0.3430
1,0.7169,0.7966,0.7433,0.3149,0.4424,0.2921,0.3405
2,0.7197,0.7975,0.7659,0.3208,0.4522,0.3040,0.3560
3,0.7163,0.7969,0.7627,0.3173,0.4482,0.2985,0.3505
4,0.7141,0.7949,0.7562,0.3145,0.4442,0.2935,0.3447
5,0.7088,0.7893,0.7249,0.3049,0.4293,0.2751,0.3216
6,0.7184,0.7946,0.7659,0.3197,0.4511,0.3024,0.3546
7,0.7194,0.7917,0.7457,0.3177,0.4456,0.2963,0.3447
8,0.7298,0.8089,0.7662,0.3304,0.4617,0.3174,0.3675
9,0.7119,0.7942,0.7532,0.3123,0.4416,0.2897,0.3407


## Evaluate Models

In [37]:
evaluate_model(lda)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

### lda Classifier

In [33]:
lda = create_model('lda', fold=10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7184,0.0000,0.7252,0.3136,0.4379,0.2874,0.3324
1,0.6923,0.0000,0.7853,0.3012,0.4354,0.2777,0.3379
2,0.7027,0.0000,0.8026,0.3119,0.4493,0.2961,0.3582
3,0.7322,0.0000,0.7163,0.3249,0.4470,0.3019,0.3430
4,0.7187,0.0000,0.7325,0.3148,0.4403,0.2904,0.3366
5,0.6512,0.0000,0.7983,0.2748,0.4088,0.2374,0.3055
6,0.7389,0.0000,0.7184,0.3318,0.4540,0.3117,0.3519
7,0.7145,0.0000,0.7435,0.3131,0.4406,0.2893,0.3382
8,0.6908,0.0000,0.8039,0.3031,0.4402,0.2827,0.3469
9,0.7243,0.0000,0.7274,0.3193,0.4438,0.2958,0.3401


## Evaluate Models

In [34]:
evaluate_model(lda)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

## Tune Model

In [36]:
tuned_lda = tune_model(lda, optimize = 'Recall')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5894,0.7260,0.7376,0.2307,0.3515,0.1580,0.2153
1,0.5916,0.7383,0.7665,0.2364,0.3614,0.1701,0.2341
2,0.5820,0.7387,0.7786,0.2340,0.3599,0.1664,0.2331
3,0.5885,0.7219,0.7581,0.2338,0.3573,0.1646,0.2266
4,0.5724,0.7101,0.7581,0.2263,0.3486,0.1513,0.2130
5,0.5836,0.7199,0.7365,0.2279,0.3480,0.1527,0.2098
6,0.5774,0.7138,0.7354,0.2248,0.3444,0.1473,0.2039
7,0.5989,0.7452,0.7851,0.2432,0.3714,0.1832,0.2513
8,0.5867,0.7262,0.7581,0.2329,0.3563,0.1631,0.2251
9,0.5981,0.7377,0.7754,0.2413,0.3680,0.1790,0.2449


In [37]:
evaluate_model(tuned_lda)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [38]:
final_lda = finalize_model(lda)

In [39]:
evaluate_model(final_lda)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

# Predict models

In [40]:
predicted_final_lda = predict_model(final_lda)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.6832,0.7599,0.7157,0.2828,0.4054,0.2413,0.2897


In [41]:
predicted_lda = predict_model(lda)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.6768,0.7571,0.7245,0.2796,0.4035,0.2375,0.2882


In [42]:
predicted_lda_tuned = predict_model(tuned_lda)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.6768,0.7571,0.7245,0.2796,0.4035,0.2375,0.2882


# The golden moment... put 10% sample test data through the models...

# Recombine test_features, test_target at first train/test/split
 This "test" set was split BEFORE Oversampling. PyCaret takes the "train" from this split and splits it internally again. Since we have no control of how PyCaret does this we needed to do a train/test split before we input the training set into PyCaret. Now we will recombine the test_features and test_target into one new df

In [43]:
test_features.shape

(11392, 22)

In [44]:
test_target.shape

(11392,)

In [45]:
test_df = test_features.copy()

In [46]:
test_df['zeroBalCode'] = test_target

In [47]:
test_df.shape

(11392, 23)

In [48]:
totalTestCount = test_df.zeroBalCode.value_counts().sum()
posTestCount = test_df.zeroBalCode.value_counts()[1]
print('#############################################')
print('This "Test" set has not been oversamplied')
print(test_df.zeroBalCode.value_counts())
print(f'Positive samples are {round(posTestCount/totalTestCount*100,2)}%')

#############################################
This "Test" set has not been oversamplied
0    9612
1    1780
Name: zeroBalCode, dtype: int64
Positive samples are 15.62%


## Create Confusion Matrix output functions

In [49]:
def calc_confusion(row):
    if ((row['zeroBalCode'] == 0) & (row['Label'] == 0)):
        value = 'TrueNegative'
    elif ((row['zeroBalCode'] == 0) & (row['Label'] == 1)):
        value = 'FalsePositive'
    elif ((row['zeroBalCode'] == 1) & (row['Label'] == 1)):
        value = 'TruePositive'
    elif ((row['zeroBalCode'] == 1) & (row['Label'] == 0)):
        value = 'FalseNegative'
    else:
        value = 'Undefined'
    return value

In [50]:
def cM(df):
    print(f'           ##############################')
    print(f'           #             #              #')
    print(f'         0 #    {df["TrueNegative"]}     #     {df["FalsePositive"]}      #')
    print(f'  True     #             #              #')
    print(f'  Class    ##############################')
    print(f'           #             #              #')
    print(f'         1 #    {df["FalseNegative"]}      #     {df["TruePositive"]}      #')
    print(f'           #             #              #')
    print(f'           ##############################')
    print(f'                  0              1        ')
    print(f'                  Predicted Class           ')


In [51]:
def cMCalc():
    # 0 or 2 = Predict equals truth, 1 = Predict doesn't equal truth
    totalNegative = confusionMatrix['TrueNegative'] + confusionMatrix['FalseNegative']
    totalPositive = confusionMatrix['TruePositive'] + confusionMatrix['FalsePositive']
    total = totalNegative + totalPositive
    trueNegativePct = confusionMatrix['TrueNegative'] / total * 100
    falseNegativePct = confusionMatrix['FalseNegative'] / total * 100
    truePositivePct = confusionMatrix['TruePositive'] / total * 100
    falsePositivePct = confusionMatrix['FalsePositive'] / total * 100

    # Accuracy Calculation
    accuracy = (confusionMatrix['TruePositive'] + confusionMatrix['TrueNegative'])/(totalPositive + totalNegative)
    # Recall Calculation
    recall = (confusionMatrix['TruePositive']/(confusionMatrix['TruePositive'] + confusionMatrix['FalseNegative']))
    # Precision Calculation
    precision = (confusionMatrix['TruePositive']/(confusionMatrix['TruePositive'] + confusionMatrix['FalsePositive']))

    print(f'TrueNegative:  {round(trueNegativePct,2)}%')
    print(f'FalseNegative:  {round(falseNegativePct,2)}%')
    print(f'TruePositive:   {round(truePositivePct,2)}%')
    print(f'FalsePositive: {round(falsePositivePct,2)}%')
    print(f'#####################################################')
    print(f'Accuracy: {round(accuracy,2)*100}%   Recall: {round(recall,2)*100}%   Precision: {round(precision,2)*100}%')
    print(f'#####################################################')

# Evaluate Models

In [52]:
predictions = predict_model(final_nb, data=test_df)

In [53]:
pd.set_option('max_columns',None)
predictions.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,avgCreditScore,loanAge,mSA,fmacRateAvg,rateDiffAvg,rateDiffAvgPct,origYear,origMonth,zeroBalCode,Label,Score
0,C,"JPMORGAN CHASE BANK, NA",6.250,146000,360,90,90,1,44,R,UT,844,25.0,1,694,66,36260,6.1125,0.1375,0.022495,2002,10,1,1,0.9031
1,R,"JPMORGAN CHASE BANK, NA",5.750,94000,360,80,80,1,26,C,LA,701,0.0,0,689,33,35380,5.4840,0.2660,0.048505,2003,5,0,0,0.1979
2,C,"CITIMORTGAGE, INC.",5.500,53000,180,15,15,2,50,C,CT,68,0.0,0,598,175,14860,5.8300,-0.3300,-0.056604,2004,4,0,0,0.0000
3,C,"GMAC MORTGAGE, LLC",5.500,134000,360,67,67,1,62,P,KY,405,0.0,0,718,62,30460,5.6950,-0.1950,-0.034241,2005,7,0,1,0.5906
4,C,"BANK OF AMERICA, N.A.",5.875,101000,360,66,66,1,34,C,CA,959,0.0,0,649,11,17020,5.9325,-0.0575,-0.009692,2003,11,0,0,0.0146


In [54]:
results = predictions[['zeroBalCode','Label','Score']]

In [55]:
predictions.Label.value_counts()

0    8164
1    3228
Name: Label, dtype: int64

In [56]:
results['Confusion'] = results.apply(calc_confusion, axis=1)

In [57]:
confusionMatrix = results.Confusion.value_counts().to_dict()

In [58]:
cM(confusionMatrix)

           ##############################
           #             #              #
         0 #    7394     #     2218      #
  True     #             #              #
  Class    ##############################
           #             #              #
         1 #    770      #     1010      #
           #             #              #
           ##############################
                  0              1        
                  Predicted Class           


In [59]:
cMCalc()

TrueNegative:  64.91%
FalseNegative:  6.76%
TruePositive:   8.87%
FalsePositive: 19.47%
#####################################################
Accuracy: 74.0%   Recall: 56.99999999999999%   Precision: 31.0%
#####################################################


In [82]:
#save_model(final_nb, 'np.final5pct_092020')